# Module 5: File System Operations

## Welcome to Module 5!

In Module 4, you mastered control flow. Now it's time to put those skills to work with **file system operations** - one of the most powerful automation capabilities in PowerShell.

### What You'll Learn

- Get-ChildItem for exploring directories
- Creating files and folders (New-Item)
- Copying items (Copy-Item)
- Moving items (Move-Item)
- Renaming items (Rename-Item)
- Deleting items (Remove-Item)
- Testing paths (Test-Path)
- File properties and attributes
- Recursive operations (-Recurse)
- Advanced filtering (by date, size, extension)
- Building a file organizer

### Why File System Operations Matter

File operations are the foundation of automation:
- **Organize** thousands of files automatically
- **Find** files based on any criteria
- **Clean up** old or temporary files
- **Backup** important data
- **Process** files in bulk

Let's become file system masters!

## Setup: Prepare Practice Environment

In [1]:
import subprocess
from pathlib import Path
import time

# Practice folder
practice_folder = Path.home() / "Documents" / "AutomationPractice"
practice_folder.mkdir(exist_ok=True)

# Create FileSystem practice subfolder
fs_practice = practice_folder / "FileSystem_Practice"
fs_practice.mkdir(exist_ok=True)

print(f"Practice folder ready: {fs_practice}")
print("Let's master file system operations!\n")

# Helper function
def run_ps(command, cwd=None):
    """Run a PowerShell command and return output."""
    if cwd is None:
        cwd = str(fs_practice)
    
    result = subprocess.run(
        ['powershell', '-Command', command],
        cwd=cwd,
        capture_output=True,
        text=True,
        timeout=30
    )
    
    return result.stdout + result.stderr

print("✓ Helper function ready!")

Practice folder ready: C:\Users\USER\Documents\AutomationPractice\FileSystem_Practice
Let's master file system operations!

✓ Helper function ready!


## 1. Get-ChildItem: Exploring the File System

The most fundamental cmdlet for file operations is **Get-ChildItem** (aliases: `gci`, `dir`, `ls`).

### Basic Get-ChildItem

In [2]:
# Create some test files first
for i in range(1, 6):
    (fs_practice / f"file{i}.txt").write_text(f"Content {i}")

# List all items
output = run_ps('''
Write-Host "=== All Items ==="
Get-ChildItem | Format-Table Name, Length, LastWriteTime -AutoSize
''')

print(output)

=== All Items ===

Name      Length LastWriteTime         
----      ------ -------------         
file1.txt      9 11/14/2025 10:39:59 PM
file2.txt      9 11/14/2025 10:39:59 PM
file3.txt      9 11/14/2025 10:39:59 PM
file4.txt      9 11/14/2025 10:39:59 PM
file5.txt      9 11/14/2025 10:39:59 PM





### Get-ChildItem with Filters

In [3]:
# Create different file types
(fs_practice / "document.docx").write_text("Document")
(fs_practice / "image.jpg").write_text("Image")
(fs_practice / "script.ps1").write_text("Script")

# Filter by extension
output = run_ps('''
Write-Host "=== Text Files Only ==="
Get-ChildItem -Filter "*.txt" | Format-Table Name, Length

Write-Host ""
Write-Host "=== All PowerShell Scripts ==="
Get-ChildItem -Filter "*.ps1" | Format-Table Name
''')

print(output)

=== Text Files Only ===

Name      Length
----      ------
file1.txt      9
file2.txt      9
file3.txt      9
file4.txt      9
file5.txt      9



=== All PowerShell Scripts ===

Name      
----      
script.ps1





### Get-ChildItem: Files vs Directories

In [4]:
# Create some directories
(fs_practice / "Folder1").mkdir(exist_ok=True)
(fs_practice / "Folder2").mkdir(exist_ok=True)
(fs_practice / "Folder3").mkdir(exist_ok=True)

output = run_ps('''
Write-Host "=== Files Only ==="
$files = Get-ChildItem -File
Write-Host "Count: $($files.Count)"
$files | Format-Table Name

Write-Host ""
Write-Host "=== Directories Only ==="
$dirs = Get-ChildItem -Directory
Write-Host "Count: $($dirs.Count)"
$dirs | Format-Table Name
''')

print(output)

=== Files Only ===
Count: 8

Name         
----         
document.docx
file1.txt    
file2.txt    
file3.txt    
file4.txt    
file5.txt    
image.jpg    
script.ps1   



=== Directories Only ===
Count: 3

Name   
----   
Folder1
Folder2
Folder3





## 2. Creating Files and Folders

Use **New-Item** to create files and directories.

### Creating Files

In [5]:
output = run_ps('''
Write-Host "=== Creating Files ==="

# Create an empty file
New-Item -Path "empty.txt" -ItemType File -Force | Out-Null
Write-Host "Created: empty.txt"

# Create a file with content
New-Item -Path "greeting.txt" -ItemType File -Value "Hello, PowerShell!" -Force | Out-Null
Write-Host "Created: greeting.txt"

# Verify
Write-Host ""
Write-Host "=== New Files ==="
Get-ChildItem -Filter "*.txt" | Where-Object {$_.Name -match "empty|greeting"} | Format-Table Name, Length
''')

print(output)

=== Creating Files ===
Created: empty.txt
Created: greeting.txt

=== New Files ===

Name         Length
----         ------
empty.txt         0
greeting.txt     18





### Creating Directories

In [6]:
output = run_ps('''
Write-Host "=== Creating Directories ==="

# Create single folder
New-Item -Path "Projects" -ItemType Directory -Force | Out-Null
Write-Host "Created: Projects"

# Create nested folders
New-Item -Path "Documents\Reports\2025" -ItemType Directory -Force | Out-Null
Write-Host "Created: Documents\Reports\2025 (nested)"

# Create multiple folders
"Photos", "Videos", "Music" | ForEach-Object {
    New-Item -Path $_ -ItemType Directory -Force | Out-Null
    Write-Host "Created: $_"
}

Write-Host ""
Write-Host "=== All Directories ==="
Get-ChildItem -Directory | Format-Table Name
''')

print(output)

<>:9: SyntaxWarning: invalid escape sequence '\R'
<>:9: SyntaxWarning: invalid escape sequence '\R'
C:\Users\USER\AppData\Local\Temp\ipykernel_7196\3519816464.py:9: SyntaxWarning: invalid escape sequence '\R'
  New-Item -Path "Documents\Reports\2025" -ItemType Directory -Force | Out-Null


=== Creating Directories ===
Created: Projects
Created: Documents\Reports?5 (nested)
Created: Photos
Created: Videos
Created: Music

=== All Directories ===

Name     
----     
Documents
Folder1  
Folder2  
Folder3  
Music    
Photos   
Projects 
Videos   





## 3. Testing Path Existence

Always check if files/folders exist before operations using **Test-Path**.

In [7]:
output = run_ps('''
Write-Host "=== Testing Path Existence ==="

$paths = @(
    "file1.txt",
    "missing.txt",
    "Projects",
    "NonExistent"
)

foreach ($path in $paths) {
    $exists = Test-Path $path
    
    if ($exists) {
        $item = Get-Item $path
        $type = if ($item.PSIsContainer) { "Directory" } else { "File" }
        Write-Host "✓ $path exists ($type)" -ForegroundColor Green
    }
    else {
        Write-Host "✗ $path does not exist" -ForegroundColor Red
    }
}
''')

print(output)

=== Testing Path Existence ===
û file1.txt exists (File)
? missing.txt does not exist
û Projects exists (Directory)
? NonExistent does not exist



## 4. Copying Files and Folders

Use **Copy-Item** to duplicate files and folders.

### Copying Files

In [8]:
output = run_ps('''
Write-Host "=== Copying Files ==="

# Copy single file
Copy-Item -Path "file1.txt" -Destination "file1_backup.txt"
Write-Host "Copied: file1.txt → file1_backup.txt"

# Copy to different location
Copy-Item -Path "file2.txt" -Destination (Join-Path "Projects" "file2_copy.txt")
Write-Host "Copied: file2.txt → Projects\file2_copy.txt"

# Copy multiple files
Copy-Item -Path "*.ps1" -Destination "Projects\"
Write-Host "Copied: All .ps1 files → Projects"

Write-Host ""
Write-Host "=== Files in Projects ==="
Get-ChildItem -Path "Projects" -File | Format-Table Name
''')

print(output)

=== Copying Files ===
Copied: file1.txt  file1_backup.txt
Copied: file2.txt  Projectsile2_copy.txt
Copied: All .ps1 files  Projects

=== Files in Projects ===

Name          
----          
file2_copy.txt
script.ps1    





### Copying Folders (Recursive)

In [9]:
# Create a folder structure to copy
(fs_practice / "SourceFolder" / "SubFolder").mkdir(parents=True, exist_ok=True)
(fs_practice / "SourceFolder" / "file_a.txt").write_text("Content A")
(fs_practice / "SourceFolder" / "SubFolder" / "file_b.txt").write_text("Content B")

output = run_ps('''
Write-Host "=== Copying Folders ==="

# Copy entire folder with contents
Copy-Item -Path "SourceFolder" -Destination "BackupFolder" -Recurse
Write-Host "Copied: SourceFolder → BackupFolder (with all contents)"

Write-Host ""
Write-Host "=== BackupFolder Contents ==="
Get-ChildItem -Path "BackupFolder" -Recurse -File | Format-Table FullName
''')

print(output)

=== Copying Folders ===
Copied: SourceFolder  BackupFolder (with all contents)

=== BackupFolder Contents ===

FullName                                                                                        
--------                                                                                        
C:\Users\USER\Documents\AutomationPractice\FileSystem_Practice\BackupFolder\file_a.txt          
C:\Users\USER\Documents\AutomationPractice\FileSystem_Practice\BackupFolder\SubFolder\file_b.txt





## 5. Moving and Renaming

**Move-Item** moves files/folders, **Rename-Item** changes names.

### Moving Files

In [10]:
output = run_ps('''
Write-Host "=== Moving Files ==="

# Create a destination folder
New-Item -Path "Archive" -ItemType Directory -Force | Out-Null

# Move single file
Move-Item -Path "file3.txt" -Destination (Join-Path "Archive" "file3.txt")
Write-Host "Moved: file3.txt → Archive"

# Move with wildcards
Move-Item -Path "*.jpg" -Destination "Photos\"
Write-Host "Moved: All .jpg files → Photos"

Write-Host ""
Write-Host "=== Archive Contents ==="
Get-ChildItem -Path "Archive" | Format-Table Name
''')

print(output)

=== Moving Files ===
Moved: file3.txt  Archive
Moved: All .jpg files  Photos

=== Archive Contents ===

Name     
----     
file3.txt





### Renaming Items

In [11]:
output = run_ps('''
Write-Host "=== Renaming Items ==="

# Rename file
Rename-Item -Path "file4.txt" -NewName "renamed_file.txt"
Write-Host "Renamed: file4.txt → renamed_file.txt"

# Rename folder
Rename-Item -Path "Folder1" -NewName "MyFolder"
Write-Host "Renamed: Folder1 → MyFolder"

Write-Host ""
Write-Host "=== Current Items ==="
Get-ChildItem | Where-Object {$_.Name -match "renamed|MyFolder"} | Format-Table Name, Mode
''')

print(output)

=== Renaming Items ===
Renamed: file4.txt  renamed_file.txt
Renamed: Folder1  MyFolder

=== Current Items ===

Name             Mode  
----             ----  
MyFolder         d-----
renamed_file.txt -a----





## 6. Deleting Files and Folders

**Remove-Item** deletes files and folders. **Be careful** - deletions are permanent!

### Deleting Files

In [12]:
# Create some temp files
for i in range(1, 4):
    (fs_practice / f"temp{i}.tmp").write_text(f"Temp {i}")

output = run_ps('''
Write-Host "=== Deleting Files ==="

# Delete single file
Remove-Item -Path "empty.txt" -Force
Write-Host "Deleted: empty.txt"

# Delete with wildcards
Remove-Item -Path "*.tmp" -Force
Write-Host "Deleted: All .tmp files"

# Verify
Write-Host ""
Write-Host "=== Remaining .tmp files ==="
$tmpFiles = Get-ChildItem -Filter "*.tmp"
if ($tmpFiles.Count -eq 0) {
    Write-Host "None (all deleted successfully)" -ForegroundColor Green
}
''')

print(output)

=== Deleting Files ===
Deleted: empty.txt
Deleted: All .tmp files

=== Remaining .tmp files ===
None (all deleted successfully)



### Deleting Folders

In [13]:
output = run_ps('''
Write-Host "=== Deleting Folders ==="

# Delete empty folder
Remove-Item -Path "Folder2" -Force
Write-Host "Deleted: Folder2 (empty)"

# Delete folder with contents (requires -Recurse)
Remove-Item -Path "BackupFolder" -Recurse -Force
Write-Host "Deleted: BackupFolder (with all contents)"

Write-Host ""
Write-Host "=== Verification ==="
$folder2Exists = Test-Path "Folder2"
$backupExists = Test-Path "BackupFolder"
Write-Host "Folder2 exists: $folder2Exists"
Write-Host "BackupFolder exists: $backupExists"
''')

print(output)

=== Deleting Folders ===
Deleted: Folder2 (empty)
Deleted: BackupFolder (with all contents)

=== Verification ===
Folder2 exists: False
BackupFolder exists: False



## 7. File Properties and Attributes

Access detailed file information using properties.

In [14]:
# Create a test file with some content
(fs_practice / "test_properties.txt").write_text("This is a test file with some content.\n" * 10)

output = run_ps('''
Write-Host "=== File Properties ==="

$file = Get-Item "test_properties.txt"

Write-Host "Name: $($file.Name)"
Write-Host "Full Path: $($file.FullName)"
Write-Host "Extension: $($file.Extension)"
Write-Host "Size: $($file.Length) bytes"
Write-Host "Created: $($file.CreationTime)"
Write-Host "Modified: $($file.LastWriteTime)"
Write-Host "Accessed: $($file.LastAccessTime)"
Write-Host "Attributes: $($file.Attributes)"
Write-Host "Is ReadOnly: $($file.IsReadOnly)"
Write-Host "Directory: $($file.DirectoryName)"
''')

print(output)

=== File Properties ===
Name: test_properties.txt
Full Path: C:\Users\USER\Documents\AutomationPractice\FileSystem_Practice\test_properties.txt
Extension: .txt
Size: 400 bytes
Created: 11/14/2025 22:40:33
Modified: 11/14/2025 22:40:33
Accessed: 11/14/2025 22:40:33
Attributes: Archive
Is ReadOnly: False
Directory: C:\Users\USER\Documents\AutomationPractice\FileSystem_Practice



### Setting Attributes

In [15]:
output = run_ps('''
Write-Host "=== Modifying Attributes ==="

# Make file read-only
$file = Get-Item "test_properties.txt"
$file.IsReadOnly = $true
Write-Host "Set test_properties.txt to ReadOnly"

# Check
$file = Get-Item "test_properties.txt"
Write-Host "Is ReadOnly: $($file.IsReadOnly)"

# Remove read-only
$file.IsReadOnly = $false
Write-Host "Removed ReadOnly attribute"

# Set hidden attribute
$file.Attributes = $file.Attributes -bor [System.IO.FileAttributes]::Hidden
Write-Host "Set Hidden attribute"

# Remove hidden
$file.Attributes = $file.Attributes -band (-bnot [System.IO.FileAttributes]::Hidden)
Write-Host "Removed Hidden attribute"
''')

print(output)

=== Modifying Attributes ===
Set test_properties.txt to ReadOnly
Is ReadOnly: True
Removed ReadOnly attribute
Set Hidden attribute
Removed Hidden attribute



## 8. Recursive Operations

The `-Recurse` parameter processes folders and all their contents.

In [16]:
# Create nested folder structure
(fs_practice / "DeepFolder" / "Level1" / "Level2").mkdir(parents=True, exist_ok=True)
(fs_practice / "DeepFolder" / "file_root.txt").write_text("Root level")
(fs_practice / "DeepFolder" / "Level1" / "file_L1.txt").write_text("Level 1")
(fs_practice / "DeepFolder" / "Level1" / "Level2" / "file_L2.txt").write_text("Level 2")

output = run_ps('''
Write-Host "=== Recursive Listing ==="

# List all files recursively
$allFiles = Get-ChildItem -Path "DeepFolder" -Recurse -File
Write-Host "Total files found: $($allFiles.Count)"
Write-Host ""

$allFiles | ForEach-Object {
    $relativePath = $_.FullName.Replace((Get-Location).Path + "\", "")
    Write-Host "  $relativePath"
}

Write-Host ""
Write-Host "=== Count by Depth ==="
Write-Host "Root level: $(( Get-ChildItem -Path 'DeepFolder' -File ).Count) files"
Write-Host "All levels: $(( Get-ChildItem -Path 'DeepFolder' -Recurse -File ).Count) files"
''')

print(output)

=== Recursive Listing ===
Total files found: 3

  \DeepFolder\file_root.txt
  \DeepFolder\Level1\file_L1.txt
  \DeepFolder\Level1\Level2\file_L2.txt

=== Count by Depth ===
Root level: 1 files
All levels: 3 files



## 9. Advanced Filtering

Combine Get-ChildItem with Where-Object for powerful filtering.

### Filter by Size

In [17]:
# Create files of different sizes
for i in range(1, 6):
    size = i * 50
    (fs_practice / f"data{i}.dat").write_text("x" * size)

output = run_ps('''
Write-Host "=== Filter by Size ==="

# Files larger than 100 bytes
Write-Host "Files > 100 bytes:"
Get-ChildItem -Filter "*.dat" | Where-Object {$_.Length -gt 100} | 
    Format-Table Name, @{Label="Size (bytes)"; Expression={$_.Length}}

Write-Host ""
Write-Host "Files between 50 and 150 bytes:"
Get-ChildItem -Filter "*.dat" | Where-Object {$_.Length -ge 50 -and $_.Length -le 150} | 
    Format-Table Name, @{Label="Size (bytes)"; Expression={$_.Length}}
''')

print(output)

=== Filter by Size ===
Files > 100 bytes:

Name      Size (bytes)
----      ------------
data3.dat          150
data4.dat          200
data5.dat          250



Files between 50 and 150 bytes:

Name      Size (bytes)
----      ------------
data1.dat           50
data2.dat          100
data3.dat          150





### Filter by Date

In [18]:
import time

# Create old file
(fs_practice / "old_file.txt").write_text("Old file")
time.sleep(2)

# Create new files
for i in range(1, 4):
    (fs_practice / f"new_file{i}.txt").write_text(f"New file {i}")

output = run_ps('''
Write-Host "=== Filter by Date ==="

# Files modified in last hour
$oneHourAgo = (Get-Date).AddHours(-1)
Write-Host "Files modified in last hour:"
Get-ChildItem -File | Where-Object {$_.LastWriteTime -gt $oneHourAgo} | 
    Select-Object -First 5 | Format-Table Name, LastWriteTime

Write-Host ""
# Files older than 10 seconds
$tenSecondsAgo = (Get-Date).AddSeconds(-10)
Write-Host "Files older than 10 seconds:"
Get-ChildItem -File | Where-Object {$_.LastWriteTime -lt $tenSecondsAgo} | 
    Select-Object -First 3 | Format-Table Name, LastWriteTime
''')

print(output)

=== Filter by Date ===
Files modified in last hour:

Name      LastWriteTime         
----      -------------         
data1.dat 11/14/2025 10:40:41 PM
data2.dat 11/14/2025 10:40:41 PM
data3.dat 11/14/2025 10:40:41 PM
data4.dat 11/14/2025 10:40:41 PM
data5.dat 11/14/2025 10:40:41 PM



Files older than 10 seconds:

Name             LastWriteTime         
----             -------------         
document.docx    11/14/2025 10:40:03 PM
file1.txt        11/14/2025 10:39:59 PM
file1_backup.txt 11/14/2025 10:39:59 PM





### Filter by Extension

In [19]:
output = run_ps('''
Write-Host "=== Filter by Extension ==="

# Group files by extension
$grouped = Get-ChildItem -File | Group-Object Extension

Write-Host "Files grouped by extension:"
$grouped | ForEach-Object {
    $ext = if ($_.Name) { $_.Name } else { "(no extension)" }
    Write-Host "  $ext : $($_.Count) files"
}

Write-Host ""
Write-Host "=== Text Files Only ==="
Get-ChildItem -File | Where-Object {$_.Extension -eq ".txt"} | 
    Select-Object -First 5 | Format-Table Name
''')

print(output)

=== Filter by Extension ===
Files grouped by extension:
  .dat : 5 files
  .docx : 1 files
  .txt : 11 files
  .ps1 : 1 files

=== Text Files Only ===

Name            
----            
file1.txt       
file1_backup.txt
file2.txt       
file5.txt       
greeting.txt    





## 10. Practical Example: File Organizer

Let's build a comprehensive file organizer that sorts files by type and size.

In [20]:
# Create mixed files to organize
(fs_practice / "OrganizeMe").mkdir(exist_ok=True)
organize_folder = fs_practice / "OrganizeMe"

# Create various files
(organize_folder / "report.pdf").write_text("PDF" * 100)
(organize_folder / "photo1.jpg").write_text("Image" * 50)
(organize_folder / "photo2.png").write_text("Image" * 50)
(organize_folder / "notes.txt").write_text("Text" * 30)
(organize_folder / "data.csv").write_text("Data" * 40)
(organize_folder / "script.py").write_text("Code" * 20)
(organize_folder / "movie.mp4").write_text("Video" * 200)
(organize_folder / "song.mp3").write_text("Audio" * 150)

output = run_ps('''
Write-Host "=== File Organizer ==="
Write-Host ""

$sourcePath = "OrganizeMe"

# Define organization rules
$categories = @{
    "Documents" = @(".pdf", ".docx", ".txt", ".csv")
    "Images" = @(".jpg", ".png", ".gif", ".bmp")
    "Videos" = @(".mp4", ".avi", ".mkv", ".mov")
    "Audio" = @(".mp3", ".wav", ".flac")
    "Code" = @(".py", ".ps1", ".js", ".java")
}

# Get all files
$files = Get-ChildItem -Path $sourcePath -File
Write-Host "Found $($files.Count) files to organize"
Write-Host ""

# Organize files
foreach ($file in $files) {
    $moved = $false
    
    foreach ($category in $categories.Keys) {
        if ($categories[$category] -contains $file.Extension) {
            # Create category folder if needed
            $destFolder = Join-Path $sourcePath $category
            if (-not (Test-Path $destFolder)) {
                New-Item -Path $destFolder -ItemType Directory -Force | Out-Null
            }
            
            # Move file
            $destPath = Join-Path $destFolder $file.Name
            Move-Item -Path $file.FullName -Destination $destPath -Force
            
            Write-Host "[${category}] $($file.Name)" -ForegroundColor Cyan
            $moved = $true
            break
        }
    }
    
    if (-not $moved) {
        Write-Host "[Other] $($file.Name)" -ForegroundColor Yellow
    }
}

Write-Host ""
Write-Host "=== Organization Complete ==="
Write-Host ""

# Show organized structure
Get-ChildItem -Path $sourcePath -Directory | ForEach-Object {
    $fileCount = (Get-ChildItem -Path $_.FullName -File).Count
    Write-Host "$($_.Name): $fileCount files"
}
''')

print(output)

=== File Organizer ===

Found 8 files to organize

[Documents] data.csv
[Videos] movie.mp4
[Documents] notes.txt
[Images] photo1.jpg
[Images] photo2.png
[Documents] report.pdf
[Code] script.py
[Audio] song.mp3

=== Organization Complete ===

Audio: 1 files
Code: 1 files
Documents: 3 files
Images: 2 files
Videos: 1 files



## 11. Practical Example: Cleanup Old Files

Build a script that finds and removes old temporary files.

In [21]:
# Create temp files with different ages
import datetime
import os

(fs_practice / "TempFiles").mkdir(exist_ok=True)
temp_folder = fs_practice / "TempFiles"

# Create "old" temp files (we'll mark them as old)
for i in range(1, 4):
    old_file = temp_folder / f"old_temp{i}.tmp"
    old_file.write_text(f"Old temp {i}")

# Wait a moment
time.sleep(1)

# Create "new" temp files  
for i in range(1, 3):
    new_file = temp_folder / f"new_temp{i}.tmp"
    new_file.write_text(f"New temp {i}")

output = run_ps('''
Write-Host "=== Cleanup Old Files ==="
Write-Host ""

$tempPath = "TempFiles"
$daysOld = 0  # For demo, use 0 days (files older than now)
$cutoffDate = (Get-Date).AddDays(-$daysOld).AddSeconds(-5)

Write-Host "Looking for .tmp files older than: $cutoffDate"
Write-Host ""

# Find old files
$oldFiles = Get-ChildItem -Path $tempPath -Filter "*.tmp" | 
    Where-Object {$_.LastWriteTime -lt $cutoffDate}

if ($oldFiles.Count -eq 0) {
    Write-Host "No old files found." -ForegroundColor Green
}
else {
    Write-Host "Found $($oldFiles.Count) old file(s):" -ForegroundColor Yellow
    
    foreach ($file in $oldFiles) {
        Write-Host "  - $($file.Name) (modified: $($file.LastWriteTime))"
    }
    
    Write-Host ""
    Write-Host "Deleting old files..."
    
    $oldFiles | Remove-Item -Force
    
    Write-Host "Deleted $($oldFiles.Count) file(s)" -ForegroundColor Green
}

Write-Host ""
Write-Host "=== Remaining Files ==="
$remaining = Get-ChildItem -Path $tempPath -Filter "*.tmp"
Write-Host "Files remaining: $($remaining.Count)"
$remaining | Format-Table Name, LastWriteTime
''')

print(output)

=== Cleanup Old Files ===

Looking for .tmp files older than: 11/14/2025 22:40:53

No old files found.

=== Remaining Files ===
Files remaining: 5

Name          LastWriteTime         
----          -------------         
new_temp1.tmp 11/14/2025 10:40:56 PM
new_temp2.tmp 11/14/2025 10:40:56 PM
old_temp1.tmp 11/14/2025 10:40:55 PM
old_temp2.tmp 11/14/2025 10:40:55 PM
old_temp3.tmp 11/14/2025 10:40:55 PM





## 12. Practical Example: Disk Usage Report

Generate a report showing disk usage by folder.

In [22]:
output = run_ps('''
Write-Host "=== Disk Usage Report ==="
Write-Host ""

# Get all folders
$folders = Get-ChildItem -Directory

# Calculate size for each folder
$folderSizes = @()

foreach ($folder in $folders) {
    $files = Get-ChildItem -Path $folder.FullName -Recurse -File -ErrorAction SilentlyContinue
    $totalSize = ($files | Measure-Object -Property Length -Sum).Sum
    
    if ($null -eq $totalSize) { $totalSize = 0 }
    
    $folderSizes += [PSCustomObject]@{
        Folder = $folder.Name
        Size = $totalSize
        Files = $files.Count
    }
}

# Sort by size (largest first)
$folderSizes = $folderSizes | Sort-Object Size -Descending

# Display report
Write-Host "Folder Usage Report:"
Write-Host "===================" 

foreach ($item in $folderSizes) {
    $sizeKB = [math]::Round($item.Size / 1KB, 2)
    Write-Host ("  {0,-20} {1,10} KB  ({2} files)" -f $item.Folder, $sizeKB, $item.Files)
}

# Total
$totalSize = ($folderSizes | Measure-Object -Property Size -Sum).Sum
$totalKB = [math]::Round($totalSize / 1KB, 2)
$totalFiles = ($folderSizes | Measure-Object -Property Files -Sum).Sum

Write-Host ""
Write-Host "Total: $totalKB KB in $totalFiles files"
''')

print(output)

=== Disk Usage Report ===

Folder Usage Report:
  OrganizeMe                 2.84 KB  (8 files)
  TempFiles                  0.05 KB  (5 files)
  DeepFolder                 0.02 KB  (3 files)
  SourceFolder               0.02 KB  (2 files)
  Projects                   0.01 KB  (2 files)
  Archive                    0.01 KB  (1 files)
  Photos                        0 KB  (1 files)
  Videos                        0 KB  (0 files)
  Folder3                       0 KB  (0 files)
  Documents                     0 KB  (0 files)
  MyFolder                      0 KB  (0 files)
  Music                         0 KB  (0 files)

Total: 2.96 KB in 22 files



## 13. Practice Exercise: Duplicate File Finder

**Challenge**: Create a script that finds duplicate files based on name.

In [23]:
# Create some duplicate files for testing
(fs_practice / "Duplicates").mkdir(exist_ok=True)
dup_folder = fs_practice / "Duplicates"

(dup_folder / "Sub1").mkdir(exist_ok=True)
(dup_folder / "Sub2").mkdir(exist_ok=True)

(dup_folder / "unique.txt").write_text("Unique")
(dup_folder / "duplicate.txt").write_text("Version 1")
(dup_folder / "Sub1" / "duplicate.txt").write_text("Version 2")
(dup_folder / "Sub2" / "duplicate.txt").write_text("Version 3")
(dup_folder / "another.txt").write_text("Another unique")

print("=== Test files created ===")
print("Your task: Find all duplicate filenames\n")

=== Test files created ===
Your task: Find all duplicate filenames



In [24]:
# Your solution here
# Hint: Use Get-ChildItem -Recurse, then Group-Object by Name

print("=== Your Solution ===")
# Write your code here

=== Your Solution ===


In [25]:
# Solution
output = run_ps('''
Write-Host "=== Duplicate File Finder ==="
Write-Host ""

$path = "Duplicates"

# Get all files recursively
$allFiles = Get-ChildItem -Path $path -Recurse -File

# Group by name
$grouped = $allFiles | Group-Object Name

# Find duplicates (groups with more than 1 file)
$duplicates = $grouped | Where-Object {$_.Count -gt 1}

if ($duplicates.Count -eq 0) {
    Write-Host "No duplicate files found." -ForegroundColor Green
}
else {
    Write-Host "Found $($duplicates.Count) filename(s) with duplicates:" -ForegroundColor Yellow
    Write-Host ""
    
    foreach ($dup in $duplicates) {
        Write-Host "File: $($dup.Name) ($($dup.Count) copies)" -ForegroundColor Cyan
        
        foreach ($file in $dup.Group) {
            $relativePath = $file.FullName.Replace((Get-Location).Path + "\", "")
            Write-Host "  - $relativePath"
        }
        
        Write-Host ""
    }
}
''')

print("=== Exercise Solution ===")
print(output)

=== Exercise Solution ===
=== Duplicate File Finder ===

Found 3 filename(s) with duplicates:

File: duplicate.txt (3 copies)
  - \Duplicates\duplicate.txt
  - \Duplicates\Sub1\duplicate.txt
  - \Duplicates\Sub2\duplicate.txt




## Summary and Key Takeaways

Congratulations! You've mastered file system operations in PowerShell.

### What You Learned:

✓ **Get-ChildItem** - Exploring files and folders  
✓ **New-Item** - Creating files and directories  
✓ **Copy-Item** - Duplicating items  
✓ **Move-Item** - Moving items  
✓ **Rename-Item** - Renaming items  
✓ **Remove-Item** - Deleting items  
✓ **Test-Path** - Checking existence  
✓ **File Properties** - Accessing metadata  
✓ **Recursive Operations** - Processing nested folders  
✓ **Advanced Filtering** - By size, date, extension  
✓ **Real-World Projects** - Organizer, cleanup, reports  

### Quick Reference: File Operations

```powershell
# List files
Get-ChildItem -Path "C:\Folder" -Recurse -File

# Create
New-Item -Path "file.txt" -ItemType File
New-Item -Path "Folder" -ItemType Directory

# Copy
Copy-Item -Path "source.txt" -Destination "dest.txt"
Copy-Item -Path "Folder" -Destination "Backup" -Recurse

# Move
Move-Item -Path "file.txt" -Destination "NewFolder\"

# Rename
Rename-Item -Path "old.txt" -NewName "new.txt"

# Delete
Remove-Item -Path "file.txt" -Force
Remove-Item -Path "Folder" -Recurse -Force

# Test existence
Test-Path "file.txt"

# Filter examples
Get-ChildItem | Where-Object {$_.Length -gt 1MB}
Get-ChildItem | Where-Object {$_.Extension -eq ".txt"}
$cutoff = (Get-Date).AddDays(-7)
Get-ChildItem | Where-Object {$_.LastWriteTime -gt $cutoff}
```

### Common Patterns

**Organize files by extension**:
```powershell
$files = Get-ChildItem -File
foreach ($file in $files) {
    $folder = $file.Extension.TrimStart('.')
    New-Item -Path $folder -ItemType Directory -Force | Out-Null
    Move-Item -Path $file -Destination "$folder\"
}
```

**Find large files**:
```powershell
Get-ChildItem -Recurse -File | 
    Where-Object {$_.Length -gt 10MB} | 
    Sort-Object Length -Descending | 
    Select-Object -First 10
```

**Backup with timestamp**:
```powershell
$timestamp = Get-Date -Format "yyyyMMdd_HHmmss"
$backupName = "Backup_$timestamp"
Copy-Item -Path "Important" -Destination $backupName -Recurse
```

### Safety Tips

1. **Always test first**: Use `-WhatIf` parameter to preview changes
   ```powershell
   Remove-Item -Path "*.tmp" -WhatIf
   ```

2. **Check before deleting**: Use Test-Path or Get-ChildItem first

3. **Use -Force carefully**: It bypasses confirmation prompts

4. **Backup important data**: Before bulk operations

5. **Test in practice folder**: Don't experiment on real data

### Next Steps

In **Module 6: Text Processing and Data Manipulation**, you'll learn:
- Reading and writing text files
- Processing CSV and JSON data
- Regular expressions for pattern matching
- Building a log file analyzer

You now have the power to automate any file-related task!

## Cleanup

Run this cell to remove all practice files:

In [26]:
import shutil

print("Cleaning up FileSystem_Practice folder...\n")

if fs_practice.exists():
    shutil.rmtree(fs_practice)
    print(f"✓ Removed {fs_practice}")
    print("\nAll practice files deleted.")
    print("The main AutomationPractice folder remains for future modules.")
else:
    print("Practice folder already cleaned up!")

Cleaning up FileSystem_Practice folder...

✓ Removed C:\Users\USER\Documents\AutomationPractice\FileSystem_Practice

All practice files deleted.
The main AutomationPractice folder remains for future modules.
